In [ ]:
import cv2
import numpy as np
import pytesseract
from mss import mss
from PIL import Image
import re
from pynput import mouse
import time

In [ ]:
# tesseract path 
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
# Function to get box location to pass to MSS
def on_click(x, y, button, pressed):
    global clicks
    if pressed:
        clicks.append((x, y))
        if len(clicks) == 2:
            return False

def get_monitor_section():
    global clicks
    clicks = []

    with mouse.Listener(on_click=on_click) as listener:
        listener.join()

    if len(clicks) != 2:
        print("Error: Did not register exactly two clicks.")
        return None

    top_left = clicks[0]
    bottom_right = clicks[1]

    monitor = {
        "top": top_left[1],
        "left": top_left[0],
        "width": abs(top_left[0] - bottom_right[0]),
        "height": abs(top_left[1] - bottom_right[1])
    }

    return monitor

In [ ]:
monitor_section = get_monitor_section()
if monitor_section:
    print(monitor_section)

In [ ]:
# use coords from above
monitor  = {'top': 866, 'left': -492, 'width': 326, 'height': 33}

# start screen
sct = mss()

while True:
    # screenshot
    sct_img = sct.grab(monitor)
    
    #to array 
    img = np.array(sct_img)
    

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    gray = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # tesseract custom - search fro numbers and / 
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789/.'
    text = pytesseract.image_to_string(gray, config=custom_config)
    print("Extracted Text:", text)

    # regex mathch pattern
    odds_pattern = re.compile(r'\d+/\d+')
    odds_matches = odds_pattern.findall(text)

    #check 2 odds?
    if len(odds_matches) >= 2:
        odds_1 = odds_matches[0]
        odds_2 = odds_matches[1]
        print(f"Odds 1: {odds_1}, Odds 2: {odds_2}")
    else:
        print("Could not find two odds values")

    # print screen, to check location 
    cv2.imshow('Screen Capture', gray)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break